In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/teacher_review_dataset.zip

Archive:  /content/drive/MyDrive/teacher_review_dataset.zip
replace teacher_review_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, log_loss
from matplotlib import pyplot as plt

In [ ]:
def get_valid_input(prompt):
    while True:
        try:
            value = int(input(prompt))
        except ValueError:
            print("Sorry, I didn't understand that.")
            continue

        if value < 0:
            print("Sorry, your response must not be negative.")
            continue
        elif value == 0:
            print("Sorry, your response must not be zero.")
            continue
        elif value > 5:
            print("Sorry, your response must not be greater than 5.")
            continue
        else:
            break
    return value

In [ ]:
# Read the dataset and drop irrelevant columns
df = pd.read_csv("teacher_review_dataset.csv")
df1 = df.drop(df.columns[[0, 1, 2]], axis=1)

In [ ]:
# Normalize the data
scaler = StandardScaler(with_mean=False)
X = scaler.fit_transform(df1.iloc[:, :-1])
y = df.iloc[:, -1].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Create the SVM classifier
clf = SVC(kernel='linear', C=100)

In [ ]:
# Train the classifier
history = clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=2).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [ ]:
# Predict the labels for the test set
y_pred = clf.predict(X_test)

In [ ]:
# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
loss = log_loss(y_test, y_pred)

In [ ]:
print("Accuracy:", accuracy)
print("Loss:", loss)

Accuracy: 0.9888888888888889
Loss: 0.40048503765685745


In [ ]:
# Get the ratings for the questions from the user
l1 = []
l2 = []
print("Enter the rating scores for the f4aculty based on the following qustions, on a scale of 1 to 5")
print("Poor: 1 Fair: 2 Good: 3, Very Good: 4, Excellent: 5")
q1 = l1.append(get_valid_input("The faculty explained the objective of the course. Its relevance in regard to Industrial application, current development and research opportunities.   "))
q2 = l1.append(get_valid_input("The prerequisites, pertinence of the course with others and programme as a whole and the organization of the subject matter are explained.   "))
q3 = l1.append(get_valid_input("The teacher explained CO statements and its correlations with the PO's and PSO's.   "))
q4 = l1.append(get_valid_input("The teacher is enthusiastic and created interest in the subject.   "))
q5 = l1.append(get_valid_input("The teacher delivered the lecture lucidly.   "))
q6 = l1.append(get_valid_input("The teacher emphasized on numerical problem solving / mathematical formulation etc, example and data analysis.   "))
q7 = l1.append(get_valid_input("Teacher used modern and smart teaching aids, whenever relevant.   "))
q8 = l1.append(get_valid_input("Test, Assignment and quizzes were adequate.   "))
q9 = l1.append(get_valid_input("The teacher provides opportunities for participatory learning.   "))
q10 = l1.append(get_valid_input("Your level of satisfaction with the all round contribution of the teacher.   "))
l2.append(l1)

Enter the rating scores for the f4aculty based on the following qustions, on a scale of 1 to 5
Poor: 1 Fair: 2 Good: 3, Very Good: 4, Excellent: 5
The faculty explained the objective of the course. Its relevance in regard to Industrial application, current development and research opportunities.   3
The prerequisites, pertinence of the course with others and programme as a whole and the organization of the subject matter are explained.   2
The teacher explained CO statements and its correlations with the PO's and PSO's.   4
The teacher is enthusiastic and created interest in the subject.   3
The teacher delivered the lecture lucidly.   3
The teacher emphasized on numerical problem solving / mathematical formulation etc, example and data analysis.   4
Teacher used modern and smart teaching aids, whenever relevant.   3
Test, Assignment and quizzes were adequate.   4
The teacher provides opportunities for participatory learning.   2
Your level of satisfaction with the all round contributi

In [ ]:
# Encode categorical data
dat_f = pd.DataFrame(l2, columns=['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10'])

In [ ]:
# Normalize the data
X1 = scaler.fit_transform(dat_f)

In [ ]:
# Predict the label
y_pred1 = clf.predict(X1)

In [ ]:
# Print the predicted label
print("The predicted label is:", y_pred1[0])

The predicted label is: 1
